# Install Packages and Imports

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
import polars as pl
import QuantLib as ql
import numpy as np
import matplotlib.pyplot as plt 
from typing import Literal
import math
import warnings
import datetime

# Load Bond Returns data and Zero Rate Curve

## Bond Returns + Data Cleaning

For the definitions of the features, refer to the pdf in the `docs` folder

Variable Summary: 

* `cs`: credit spread computed as bond_yield in excess of duration-matched portfolio of US Treasuries yields
* `tmt`: time to maturity (in months, I guess)
* `ind_num_17`: Fama-French 17 Industry Classification (may be useful for value factor)
* `size_ig`, `size_jk`: dummy for (respectively) IG/HY bonds in the BAML/ICE index
* `bond_type`: US Corporate Convertible (CCOV), US Corporate Debentures (CDEB), US Corporate Medium Term Note (CMTN), US Corporate Medium Term Note Zero (CMTZ), or US Corporate Paper (CP)
* `R_FR`, `N_FR` and co.: rating as names and one-hot encoded, probably from different providers (SP may be S&P, FR Fitch Ratings, MR Moody's
* `INTEREST_FREQUENCY`: e.g. 13 for variable coupon, -1 for NA, 14 for bi-monthly, 15-16 undocumented
* `DATED_DATE`: date from which the bond interest accrues
* Additional Info on variables: FISD data dictionary 2012 document

Prices Variables

* `PRICE_EOM`: considers all trading days and takes the last
* `PRICE_LDM`: consider only last trading day of the month otherwise NaN
* `PRICE_L5M`: consider only last 5 trading days of the month otherwise NaN
* `T_SPREAD`: average trade-weighted bid-ask spread
* `T_YLD_PT`: average trade-weighted yield point
* `T_VOLUME`: volume traded during the month, par-value
* `T_DVOLUME`: volume traded during the month, dollar value
* `bondprc` is adjusted for MMN, `BONDPRC` is unadjusted

Other notes

* We may want to remove defaulted bonds (check if they were actually already removed)

In [ ]:
bond_data = pl.read_csv('../data/bond_data_final.csv', try_parse_dates=True)

bond_data.columns

In [ ]:
# merge duration across the two dataframes

bond_data = bond_data.with_columns((pl.col('DURATION_y').fill_null(pl.col('DURATION_x'))).alias('DURATION'))
bond_data = bond_data.drop(['DURATION_x', 'DURATION_y'])

bond_data.head(1)

Remove the columns we know we don't need

In [ ]:
useless_cols = [
    'company_symbol', # we do not need it
    'TREASURY_MATURITY', # we do not need it
    'CONV', # convertible bonds (we already removed all convertibles) 
    'sic_code', # SIC Industry Code (we don't need it) 
    'mom6_1', # 6m momentum (we don't need it) 
    'ltrev48_12', # sum of bond returns over 48months (momentum) 
    'TMT', # time to maturity in years (we have it in months in tmt) and TMT has NaNs while tmt does not
    'DATE', # has NaNs while date does not
    'CUSIP', # has Nans while cusip does not
    'COUPON', # repeated in coupon and COUPON has NaNs while coupon does not
]

bond_data = bond_data.drop(useless_cols)

Some data is missing in some dates observations, but it is the same for all bonds. So, we fill the `null` values using the other date observations

In [ ]:
# fill all DATED_DATE, OFFERING_DATE, MATURITY, and DAY_COUNT_BASIS since they are all the same for each issue
cols_to_fill = ['DATED_DATE', 'OFFERING_DATE', 'DAY_COUNT_BASIS', 'MATURITY', 'NCOUPS', 'FIRST_INTEREST_DATE', 'DEFAULTED']
grouped = bond_data.group_by('cusip')

def fill_dates(group): 
    return group.with_columns(group.select(cols_to_fill).fill_null(strategy='forward').fill_null(strategy='backward'))
    
filled_data = grouped.map_groups(fill_dates).drop_nulls(subset=['DATED_DATE', 'OFFERING_DATE', 'MATURITY', 'NCOUPS'])
n_dropped_cusips = bond_data.n_unique('cusip') - filled_data.n_unique('cusip')

print(f'Removed {n_dropped_cusips} CUSIPs')
bond_data = filled_data

In [ ]:
bond_data.select(cols_to_fill + ['coupon', 'date']).null_count()

Remove the bonds which defaulted (`DEFAULTED=="N"`), since we don't want to consider distressed debt. The column is the same for each cusip and indicates if the bond defaulted at any time from offering date through maturity

In [ ]:
bond_data.select('DEFAULTED').unique()

In [ ]:
bond_data = bond_data.filter(pl.col('DEFAULTED') == 'N')

Remove extra long maturities (>30 years) and bonds maturing in <1y

In [ ]:
mask_long_maturities = pl.col('tmt') / 12 < 30
bond_data.filter(mask_long_maturities).shape[0] / bond_data.shape[0]

In [ ]:
bond_data = bond_data.filter(mask_long_maturities)

In [ ]:
mask_short_maturities = pl.col('tmt') > 12
bond_data.filter(mask_short_maturities).shape[0] / bond_data.shape[0]

In [ ]:
bond_data = bond_data.filter(mask_short_maturities)

Remove zero coupon bonds

In [ ]:
mask_zcb = pl.col('NCOUPS') > 0

new_bond_data = bond_data.filter(mask_zcb)

removed_cusips = bond_data.n_unique('cusip') - new_bond_data.n_unique('cusip')
print(f'Removed {removed_cusips} CUSIPs')

bond_data = new_bond_data

Fix the outlier where the DATED_DATE is after the first coupon

In [ ]:
bond_data.select(pl.col('FIRST_INTEREST_DATE')).null_count()

In [ ]:
bond_data_fixed = bond_data.with_columns(
    DATED_DATE = pl.when((pl.col('FIRST_INTEREST_DATE') < pl.col('DATED_DATE'))).then(pl.col('OFFERING_DATE')).otherwise(pl.col('DATED_DATE'))
)

bond_data = bond_data_fixed
to_fix_mask = pl.col('FIRST_INTEREST_DATE') < pl.col('OFFERING_DATE')

while bond_data.filter(to_fix_mask).n_unique('cusip') > 0:      
    bond_data = bond_data.with_columns(
        FIRST_INTEREST_DATE=pl
            .when(to_fix_mask & (pl.col('NCOUPS') == 2)).then(pl.col('FIRST_INTEREST_DATE').dt.offset_by('6mo'))
            .when(to_fix_mask & (pl.col('NCOUPS') == 4)).then(pl.col('FIRST_INTEREST_DATE').dt.offset_by('3mo'))
            .when(to_fix_mask & (pl.col('NCOUPS') == 12)).then(pl.col('FIRST_INTEREST_DATE').dt.offset_by('1mo'))
            .otherwise(pl.col('FIRST_INTEREST_DATE'))
    )

In [ ]:
bond_data.select(pl.col('FIRST_INTEREST_DATE')).null_count()

In [ ]:
bond_data.filter(pl.col('FIRST_INTEREST_DATE') < pl.col('OFFERING_DATE')).n_unique('cusip')

Some more overview of the data

In [ ]:
bond_data.head()

In [ ]:
bond_data.select('bondprc').null_count() / bond_data.shape[0]

### Removing Distressed debt

We remove the distressed debt outliers for OAS when we compute the signal. We just mark distressed with a new column, which we will not invest in, but we keep the data for later use in the backtest


We remove all bonds whose price goes below $70

Now two options: 
* We remove all observations after the price was hit
* If it recovers we can start re-investing in it

In [ ]:
mask = (pl.col('oas') > 0.2) & (pl.col('oas').is_not_nan())
new_bond_data.filter(mask).n_unique('cusip')

In [ ]:
mask_price = (pl.col('bondprc') < 70) & (pl.col('bondprc').is_not_nan())
new_bond_data.filter(mask).n_unique('cusip')

In [ ]:
new_bond_data.n_unique('cusip')

In [ ]:
new_bond_data.group_by('cusip').n_unique().filter(pl.col('bondprc') > 36)

In [ ]:
new_bond_data.filter((pl.col('oas') > 1) & (pl.col('oas').is_not_nan())).group_by('DEFAULTED').n_unique()

# Load the historical zero curve

In [ ]:
yield_curve = pl.read_csv('../data/yield_panel_monthly_frequency_daily_maturity.csv', try_parse_dates=True)
yield_curve = yield_curve.drop('MAX_DATA_TTM')

In [ ]:
yield_curve = yield_curve.with_columns(pl.col('').alias('date')).drop('')
yield_curve = yield_curve.with_columns(pl.all().exclude('date').cast(float))
yield_curve[:3, -3:]

In [ ]:
yield_curve = yield_curve.sort('date').upsample(
    time_column='date',
    every='1d', 
).select(pl.all().forward_fill())

In [ ]:
yield_curve = yield_curve.group_by(pl.col('date').dt.month_end()).agg(pl.all().last()).sort('date')

In [ ]:
yield_curve[:10, :10]

# OAS Calculation with QuantLib

OAS is the spread that added to the zero rates in the pricing function returns the price of the bond. We use the Newton method to get a solution for the OAS. In our case, since we stripped bonds with optionality, the OAS is the Z-Spread

In [ ]:
def decompose_date(date: datetime.date):
    """
    Returns day, month, year given a `pd.Timestamp`
    Parameters
    ----------
    date: pd.Timestamp

    Returns
    -------
    tuple[int, int, int]: day, month, year

    """
    return date.day, date.month, date.year

def get_day_count(bond: dict): 
    day_count_convention = bond['DAY_COUNT_BASIS']
    
    if day_count_convention == '30/360': 
        return ql.Thirty360(ql.Thirty360.USA)
    elif day_count_convention == 'ACT/360': 
        return ql.Actual360()
    elif day_count_convention == 'ACT/ACT': 
        return ql.ActualActual(ql.ActualActual.Bond)
    
    raise Exception(f'we did not implement day count {day_count_convention}')
    
def get_coupon_freq(bond: dict): 
    coupon_freq = bond['NCOUPS']
    if coupon_freq == 1: 
        return ql.Period(ql.Annual)
    elif coupon_freq == 2: 
        return ql.Period(ql.Semiannual)
    elif coupon_freq == 4: 
        return ql.Period(ql.Quarterly)
    elif coupon_freq == 12: 
        return ql.Period(ql.Monthly)
    
    raise Exception(f'we did not implement coupon freq {coupon_freq}')

In [ ]:
def get_zero_curve(date: ql.Date | datetime.date) -> ql.ZeroCurve: 
    # get the zero rates for that specific date
    if isinstance(date, ql.Date):
        date_polar = date.to_date()
    elif isinstance(date, datetime.date): 
        date_polar = date
    else: 
        raise Exception('unknown type for date,', type(date))
    
    zero_rates = yield_curve.row(by_predicate=(pl.col('date') == date_polar))
    if not isinstance(zero_rates[0], datetime.date): 
        raise Exception('we are cutting yields from the calculation')
    zero_rates = list(zero_rates[1:])
    # create the list of tenors based on the number of observations
    tenors = np.arange(0, len(zero_rates) + 1)
    
    # set the tenor unit and compounding frequency based on the type of data used
    tenor_unit = ql.Days
    
    # create the list of spot dates and rates
    #   (need to add a point for the evaluation date, hence the 0.)
    spot_dates = [date + ql.Period(tenor.item(), tenor_unit) for tenor in tenors] 
    spot_rates = [0.] + zero_rates
    
    # set payment convention as specified in the paper (365 days)
    pmt_convention = ql.Actual365Fixed(ql.Actual365Fixed.Standard)
    
    # create the ZeroCurve and return it
    calendar = ql.UnitedStates(ql.UnitedStates.SOFR)
    spot_curve = ql.ZeroCurve(spot_dates, spot_rates, pmt_convention, calendar, ql.Linear(), ql.Compounded, ql.Continuous)
    
    return spot_curve

def map_zero_curves(zero_curve: pl.Series): 
    date = zero_curve[0]
    
    if not isinstance(zero_curve[0], datetime.date): 
        raise Exception('we are cutting yields from the calculation')
    
    zero_rates = list(zero_curve[1:])
    # create the list of tenors based on the number of observations
    tenors = np.arange(0, len(zero_rates) + 1)
    
    # set the tenor unit and compounding frequency based on the type of data used
    tenor_unit = ql.Days
    
    # create the list of spot dates and rates
    #   (need to add a point for the evaluation date, hence the 0.)
    spot_dates = [ql.Date(date.day, date.month, date.year) + ql.Period(tenor.item(), tenor_unit) for tenor in tenors] 
    spot_rates = [0.] + zero_rates
    
    # set payment convention as specified in the paper (365 days)
    pmt_convention = ql.Actual365Fixed(ql.Actual365Fixed.Standard)
    
    # create the ZeroCurve and return it
    calendar = ql.UnitedStates(ql.UnitedStates.SOFR)
    spot_curve = ql.ZeroCurve(spot_dates, spot_rates, pmt_convention, calendar, ql.Linear(), ql.Compounded, ql.Continuous)
    
    return date, spot_curve
    


## Cache the Zero Curves for each EoM in a dictionary so to not recompute them every time

In [ ]:
# get zero curves for all days
zero_curves = dict()
for data in yield_curve.filter(pl.col('date') >= datetime.date(2000,1,1)).iter_rows(): 
    date, curve = map_zero_curves(data)
    zero_curves[date] = curve   

In [ ]:
len(zero_curves)

## Final Function for OAS Calculation

In [ ]:
def debug_cashflows(bond: ql.FixedRateBond, bond_data: pl.Series, mkt_price: float, z_spread: float, impl_clean_price: float):
    """
    Debug cashflows given a bond and bond_data. 
    
    Function to debug the results of the OAS calcuations.
    
    Parameters
    ----------
    bond
    bond_data
    """
    cashflows = bond.cashflows()
    print('--- BOND SETUP & CALCS CHECKS ---')
    print(f'\tCalc Date = {bond_data.date}, \n\tOffering date = {bond_data.OFFERING_DATE}, Maturity = {bond_data.MATURITY}')
    
    # check for coupon_amt
    data_coupon_amt = bond_data.coupon * 100 / bond_data.NCOUPS / 100 # todo account for the coupon frequency
    bond_ql_coup_amt = np.round(cashflows[2].amount(), 2)
    print(f'\tCoupon Check: Data = {data_coupon_amt}, Model = {bond_ql_coup_amt}')
        
    # check that Accrued Interest
    data_accrued_interest = bond_data.COUPACC
    bond_ql_accr_interest = np.round(bond.dirtyPrice() - bond.cleanPrice(), 2)
    print(f'\tAccrued Interest Check: Data = {data_accrued_interest}, Model = {bond_ql_accr_interest}')
    
    print('\tCASHFLOWS SCHEDULE')
    for c in cashflows:
        print('\t%20s %12f' % (c.date(), c.amount()))
        
    coupons = [ql.as_coupon(c) for c in bond.cashflows()[:-1]]
    coupons_df = pd.DataFrame([(c.date().to_date(), c.rate(), c.accrualPeriod()) for c in coupons], columns=['date', 'rate', 'accrual_period'], index=range(1,len(coupons)+1))
    print(coupons_df)
    
    # checks for coupon dates
    bond_first_pmt_date = bond_data.FIRST_INTEREST_DATE.date()
    bond_last_pmt_date = bond_data.LAST_INTEREST_DATE.date()
    bond_ql_first_pmt_date = cashflows[0].date().to_date()
    bond_ql_last_pmt_date = cashflows[-3].date().to_date()
    
    first_delta = (bond_first_pmt_date - bond_ql_first_pmt_date).days
    last_delta = (bond_last_pmt_date - bond_ql_last_pmt_date).days
    
    print('\tChecks for Payment Dates')
    print(f'\t\tFirst pmt: Data = {bond_first_pmt_date}, Model = {bond_ql_first_pmt_date}, Delta = {first_delta}')
    print(f'\t\tLast pmt: Data = {bond_last_pmt_date}, Model = {bond_ql_last_pmt_date}, Delta = {last_delta}')
    
    
    delta_p = mkt_price - impl_clean_price
    delta_bps = delta_p / mkt_price * 100 * 100

    print(f'\tZ-SPREAD = {z_spread:.5f} ({z_spread * 100:.3f}%)')
    print(f'\tMkt Price = {mkt_price}, Implied Clean Price = {impl_clean_price:.5f}, Delta = {delta_p:.5f}, Delta (bps): {delta_bps:.2f}')
    
    print(f'DEBUG: {data_coupon_amt} {bond_ql_coup_amt}')
    assert math.isclose(data_coupon_amt, bond_ql_coup_amt, rel_tol=1e-2)
    if not math.isclose(data_accrued_interest, bond_ql_accr_interest): 
        warnings.warn('Accrued Interest is not correct')
    # assert math.isclose(data_accrued_interest, bond_ql_accr_interest) 
    assert bond_first_pmt_date == bond_ql_first_pmt_date # check the first payment date matches
    print('--- ALL CHECKS PASSED FOR BOND CALCULATIONS ---')
    print('--- CHECKS FOR Z-SPREAD CALCULATIONS ---')
    assert abs(first_delta) < 3
    assert abs(last_delta) < 3
    # delta p less than 1bp
    assert abs(delta_p) < 0.05
    print('--- ALL CHECKS PASSED FOR Z-SPREAD CALCULATION ---')
    
class ParameterNaNException(Exception):
    def __init__(self, varname: str):
        self.msg = f'Variable {varname} is NaN, and it is required.'
        super().__init__(self.msg)
        
def check_parameters(bond: pl.Series): 
    for varname in ['coupon', 'principal_amt']: 
        if np.isnan(bond[varname]): raise ParameterNaNException(varname)
    
def compute_OAS(bond: pl.Series, debug: bool = False):
    # check that parameters are defined
    print(f'computing OAS for bond {bond['cusip']} at {bond['date']}...', end='')
    if bond['bondprc'] is None: 
        print('No price data, skipping this row')
        return np.nan
    check_parameters(bond)
    
    calc_date = ql.Date(*decompose_date(bond['date']))
    ql.Settings.instance().evaluationDate = calc_date
    
    # key data
    calendar = ql.UnitedStates(ql.UnitedStates.NYSE) # calendar to follow for calculations
    calendar = ql.NullCalendar()
    # day_count_convention = get_day_count(bond) # the day count convention as specified in the bond
    day_count_convention = ql.ActualActual(ql.ActualActual.Bond) # the day count convention as specified in the bond
    
    # bond data
    issue_date = ql.Date(*decompose_date(bond['OFFERING_DATE']))
    accruing_start_date = ql.Date(*decompose_date(bond['DATED_DATE'])) # this is the date from which the bond starts accruing interest
    maturity_date = ql.Date(*decompose_date(bond['MATURITY']))
    tenor = get_coupon_freq(bond)
    date_generation = ql.DateGeneration.Backward
    month_end = False
    face_value = bond['principal_amt']
    face_value = 100
    coupon = bond['coupon'] / 100
    mkt_price = bond['bondprc']
    first_pmt_date = ql.Date(*decompose_date(bond['FIRST_INTEREST_DATE']))
    
    schedule = ql.Schedule(accruing_start_date, maturity_date, tenor, calendar, ql.Unadjusted, ql.Unadjusted, date_generation, month_end, first_pmt_date)
    
    settlement_days = 0
    
    # zero curve
    spot_curve = zero_curves[bond['date']]
    pricing_curve = ql.YieldTermStructureHandle(spot_curve)
    
    bond_ql = ql.FixedRateBond(
        settlement_days, 
        face_value, 
        schedule, 
        [coupon],
        day_count_convention
    )
    bond_ql.setPricingEngine(ql.DiscountingBondEngine(pricing_curve))
    
    # Z-spread calculation 
    z_spread = ql.BondFunctions.zSpread(
        bond_ql, 
        mkt_price,
        spot_curve,
        day_count_convention, 
        ql.Compounded,
        ql.Continuous, 
        calc_date,
        1.e-16,
        10_000_000,
        0.
    )
    
    def get_impl_clean_price(spread):
        spread1 = ql.SimpleQuote(spread)
        spread_handle1 = ql.QuoteHandle(spread1)
        ts_spreaded1 = ql.ZeroSpreadedTermStructure(pricing_curve,
                                                    spread_handle1,
                                                    ql.Compounded,
                                                    ql.Continuous)
        ts_spreaded_handle1 = ql.YieldTermStructureHandle(ts_spreaded1)
        fixed_rate_bond = ql.FixedRateBond(settlement_days,
                                        face_value,
                                        schedule,
                                        [coupon],
                                        day_count_convention)
        # Set Valuation engine
        bond_engine = ql.DiscountingBondEngine(ts_spreaded_handle1)
        fixed_rate_bond.setPricingEngine(bond_engine)
        value = fixed_rate_bond.cleanPrice()
        print(f'bond NPV: {fixed_rate_bond.NPV()}, clean: {fixed_rate_bond.cleanPrice()}')
        return value
    
    if debug: 
        impl_clean_price = get_impl_clean_price(z_spread)
        debug_cashflows(bond_ql, bond, mkt_price, z_spread, impl_clean_price)
        
    print(f' ...Z-spread is {z_spread}')

    return z_spread

Compute OAS for a Random Bond

In [ ]:
example_bond = bond_data.row(10, named=True)
spot_crv = compute_OAS(example_bond)

## Run the function on the whole DataFrame

In [ ]:
bond_data = bond_data.sort(['date', 'cusip'])

In [ ]:
new_bond_data = bond_data.with_columns(oas=pl.Series(compute_OAS(bond) for bond in bond_data.iter_rows(named=True)))

In [ ]:
bond_data = new_bond_data

In [ ]:
bond_data.write_parquet('../data/bond_data_with_oas.pq', compression='zstd', compression_level=5)